In [30]:
%pip install -qU FlagEmbedding~=1.2.11 pymilvus pymilvus[model]

Note: you may need to restart the kernel to use updated packages.


### Definition Dataset exploration

In [5]:
import polars as pl
from pathlib import Path
import pickle

pl.Config.set_tbl_rows(200)
pl.Config.set_fmt_str_lengths(300)

polars.config.Config

In [6]:
dataset_path = Path('../data/definitions')

df = pl.read_csv(dataset_path / '*.tsv', separator='\t')
df.select(pl.len())

len
u32
14729


In [7]:
df.head(3)

label,definendum,definiens,full_definition,references,provenance,document
str,str,str,str,str,str,str
"""#georeferenziazione""","""georeferenziazione""","""tecnica di attribuzione di coordinate geografiche a un oggetto grafico, usata nelle procedure di cartografia computerizzata e nella costruzione di basi cartografiche digitali;""","""a) georeferenziazione : tecnica di attribuzione di coordinate geografiche a un oggetto grafico, usata nelle procedure di cartografia computerizzata e nella costruzione di basi cartografiche digitali;""","""[]""","""PDL""","""18PDL0001470_PD.xml"""
"""#sistemaInformativoGeografico""","""sistema informativo geografico""","""sistema informatico, hardware e software, utilizzato per la memorizzazione e per il processamento di dati geografici nelle modalità grafica e analitica.""","""b) sistema informativo geografico : sistema informatico, hardware e software, utilizzato per la memorizzazione e per il processamento di dati geografici nelle modalità grafica e analitica.""","""[]""","""PDL""","""18PDL0001470_PD.xml"""
"""#gestoreDelSottoprodottoUmido(Gsu)""","""gestore del sottoprodotto umido (GSU)""","""il soggetto pubblico o privato che gestisce i sottoprodotti organici ai fini e con le modalità stabiliti dalla presente legge;""","""a) gestore del sottoprodotto umido (GSU) : il soggetto pubblico o privato che gestisce i sottoprodotti organici ai fini e con le modalità stabiliti dalla presente legge;""","""[]""","""PDL""","""18PDL0001550_PD.xml"""


In [8]:
full_def_df = df.with_columns(
    pl.when(
        pl.col("definendum").is_null() | pl.col("definiens").is_null()
    ).then(
        pl.col("full_definition")
    ).otherwise(
        pl.concat_str(
            [
                pl.col("definendum"),
                pl.col("definiens"),
            ],
            separator=": ",
        )
    ).alias("joined_definition")
)

In [9]:
full_def_df.select(pl.len())

len
u32
14729


In [10]:
(
    full_def_df
    .sort(pl.col('full_definition').str.len_chars(), descending=True)
    .with_columns(
        pl.col('full_definition').str.len_chars().alias('length')
    )
).head(7)


label,definendum,definiens,full_definition,references,provenance,document,joined_definition,length
str,str,str,str,str,str,str,str,u32
"""#impresaFerroviaria""","""impresa ferroviaria""","""qualsiasi impresa pubblica o privata titolare di una licenza, la cui attivita' principale consiste nella prestazione di servizi per il trasporto sia di merci sia di persone per ferrovia e che garantisce obbligatoriamente la trazione; sono comprese anche le imprese che forniscono solo la trazione; b)…","""a) impresa ferroviaria : qualsiasi impresa pubblica o privata titolare di una licenza, la cui attivita' principale consiste nella prestazione di servizi per il trasporto sia di merci sia di persone per ferrovia e che garantisce obbligatoriamente la trazione; sono comprese anche le imprese che fornis…","""['/akn/it/act/legge/stato/1990-10-10/287/!main#art_7', '/akn/it/act/decretoLegge/stato/2011-12-06/201/!main#art_37', '/akn/it/act/legge/stato/2011-12-22/214/!main', '/akn/it/act/decretoLegge/stato/2012-01-24/1/!main#art_36', '/akn/it/act/legge/stato/2012-03-24/27/!main', '/akn/eu/act/directive/2012/…","""Normattiva""","""20150724_15G00126_VIGENZA_20220630.xml""","""impresa ferroviaria: qualsiasi impresa pubblica o privata titolare di una licenza, la cui attivita' principale consiste nella prestazione di servizi per il trasporto sia di merci sia di persone per ferrovia e che garantisce obbligatoriamente la trazione; sono comprese anche le imprese che forniscono…",12881
"""#finanziamentoDelTerrorismoRelativamenteAlloScambioDiInformazioniConLeAutoritaCompetentiDiAltriStatiMembri""","""finanziamento del terrorismo, relativamente allo scambio di informazioni con le autorita' competenti di altri Stati membri""","""la fornitura o la raccolta di capitali intenzionalmente compiuta, in qualsiasi modo, direttamente o indirettamente, con l'intenzione che tali capitali siano utilizzati, o nella consapevolezza che saranno utilizzati, in tutto o in parte, per commettere o per contribuire alla commissione di uno dei se…","""l) finanziamento del terrorismo, relativamente allo scambio di informazioni con le autorita' competenti di altri Stati membri : la fornitura o la raccolta di capitali intenzionalmente compiuta, in qualsiasi modo, direttamente o indirettamente, con l'intenzione che tali capitali siano utilizzati, o n…","""[]""","""Normattiva""","""20211129_21G00195_ORIGINALE.xml""","""finanziamento del terrorismo, relativamente allo scambio di informazioni con le autorita' competenti di altri Stati membri: la fornitura o la raccolta di capitali intenzionalmente compiuta, in qualsiasi modo, direttamente o indirettamente, con l'intenzione che tali capitali siano utilizzati, o nella…",12439
"""#organizzazioneResponsabileDellApplicazioneSullUomo""","""organizzazione responsabile dell'applicazione sull'uomo""","""una struttura sanitaria o un'unita' ospedaliera che esegue applicazioni sull'uomo di tessuti e cellule;""","""m) organizzazione responsabile dell'applicazione sull'uomo : una struttura sanitaria o un'unita' ospedaliera che esegue applicazioni sull'uomo di tessuti e cellule; ((m-bis) ""codice unico europeo"" o Single European Code (SEC): il codice unico d'identificazione applicato ai tessuti e alle cellule dis…","""[]""","""Normattiva""","""20100218_010G0030_VIGENZA_20191129.xml""","""organizzazione responsabile dell'applicazione sull'uomo: una struttura sanitaria o un'unita' ospedaliera che esegue applicazioni sull'uomo di tessuti e cellule;""",5977
"""#criminalActivity""","""criminal activity""","""means any kind of criminal involvement in the commission of any offence punishable, in accordance with national law, by deprivation of liberty or a detention order for a maximum of more than one year or, as regards Member States that have a minimum threshold for offences in their legal systems, any …","""(1) "" criminal activity "" means any kind of criminal involvement in the commission of any offence punishable, in accordance with national law, by depri

In [11]:
# drop the longest definitions

full_def_df = full_def_df.filter(pl.col('full_definition').str.len_chars() < 5000)

defs = full_def_df['joined_definition'].to_list()

In [12]:
with open('definitions_list.pkl', 'wb') as f:
    pickle.dump(defs, f)

In [13]:
full_def_df.select(
    pl.col('joined_definition'),
    pl.col('provenance'),
    pl.col('document'),
    pl.col('references'),
).with_columns(
    pl.col('references').map_elements(eval, return_dtype=pl.List(pl.String)),
).with_columns(
    pl.col('references').list.len().alias('ref_len'),
).sort(pl.col('ref_len'), descending=True).head()

joined_definition,provenance,document,references,ref_len
str,str,str,list[str],u32
"""fishing licence: means a licence as defined in point (9) of Article 4 of Council Regulation (EC) No 1224/2009 Council Regulation (EC) No 1224/2009 of 20 November 2009 establishing a Community control system for ensuring compliance with the rules of the common fisheries policy, amending Regulations (…","""EurLex""","""32013R1380.xml""","[""/akn/eu/act/regulation/ep/2009/1224/~art_4"", ""/akn/eu/act/regulation/ep/2009/1224/"", … ""/akn/eu/documentCollection/L/gu/2009-12-22/343/!main#eop_1""]",19
"""supervised entity: means any of the following: a credit institution as defined in point (1) of Article 4(1) of Regulation (EU) No 575/2013 of the European Parliament and of the Council an investment firm as defined in point (1) of Article 4(1) of Directive 2014/65/EU; an insurance undertaking as def…","""EurLex""","""32016R1011.xml""","[""/akn/eu/act/regulation/ep/2013/575/~art_4(1)"", ""/akn/eu/act/regulation/ep/2013/575/"", … ""/akn/eu/act/regulation/ep/2012/648/""]",19
"""operator: means a natural or legal person as defined in Article 4(19) of Council Regulation (EC) No 1224/2009 Council Regulation (EC) No 1224/2009 of 20 November 2009 establishing a Union control system for ensuring compliance with the rules of the common fisheries policy, amending Regulations (EC) …","""EurLex""","""32017R0218.xml""","[""/akn/eu/act/regulation/ep/2009/1224/~art_4(19)"", ""/akn/eu/act/regulation/ep/2009/1224/"", … ""/akn/eu/documentCollection/L/gu/2009-12-22/343/!main#eop_1""]",19
"""control and inspection: means any measures taken by Member States, in particular pursuant to Articles 5, 11, 71, 91 and 117 and Title VII of Council Regulation (EC) No 1224/2009 Council Regulation (EC) No 1224/2009 of 20 November 2009 establishing a Union control system for ensuring compliance with …","""EurLex""","""32019R0473.xml""","[""/akn/eu/act/regulation/ep/2009/1224/"", ""/akn/eu/act/regulation/ep/2009/1224/"", … ""/akn/eu/documentCollection/L/gu/2009-12-22/343/!main#eop_1""]",18
"""Union funds: means the European Structural and Investment Funds referred to in Article 1 of Regulation (EU) No 223/2014 of the European Parliament and of the Council Regulation (EU) No 223/2014 of the European Parliament and of the Council of 11 March 2014 , on the Fund for European Aid to the Most …","""EurLex""","""32017R0825.xml""","[""/akn/eu/act/regulation/ep/2014/223/~art_1"", ""/akn/eu/act/regulation/ep/2014/223/"", … ""/akn/eu/documentCollection/L/gu/2014-05-20/150/!main#eop_143""]",17


---

In [10]:
with open('definitions_list.pkl', 'wb') as f:
    pickle.dump(defs, f)

### Store embeddings in MilvusDB

In [1]:
from pymilvus import MilvusClient, connections, utility, FieldSchema, CollectionSchema, DataType, Collection
from pymilvus.model.hybrid import BGEM3EmbeddingFunction
import pickle

In [2]:
# setup embedding model

ef = BGEM3EmbeddingFunction(use_fp16=False, device="cpu")
dense_dim = ef.dim["dense"]

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [3]:
#defs_embeddings = ef(defs)
with open('../data/def_embeddings.pkl', 'rb') as f:
    defs_embeddings = pickle.load(f)

with open('../data/definitions_list.pkl', 'rb') as f:
    defs_list = pickle.load(f)

In [14]:
final_df = full_def_df.select(
    pl.col('joined_definition').alias('definition_text'),
    pl.col('provenance').alias('dataset'),
    pl.col('document').alias('document_id'),
    pl.col('references'),
).with_columns(
    pl.col('references').map_elements(eval, return_dtype=pl.List(pl.String)),
).with_row_index('id')

final_df.head()

id,definition_text,dataset,document_id,references
u32,str,str,str,list[str]
0,"""georeferenziazione: tecnica di attribuzione di coordinate geografiche a un oggetto grafico, usata nelle procedure di cartografia computerizzata e nella costruzione di basi cartografiche digitali;""","""PDL""","""18PDL0001470_PD.xml""",[]
1,"""sistema informativo geografico: sistema informatico, hardware e software, utilizzato per la memorizzazione e per il processamento di dati geografici nelle modalità grafica e analitica.""","""PDL""","""18PDL0001470_PD.xml""",[]
2,"""gestore del sottoprodotto umido (GSU): il soggetto pubblico o privato che gestisce i sottoprodotti organici ai fini e con le modalità stabiliti dalla presente legge;""","""PDL""","""18PDL0001550_PD.xml""",[]
3,"""scarti alimentari: sottoprodotti della produzione, della lavorazione e della preparazione di prodotti alimentari;""","""PDL""","""18PDL0001550_PD.xml""",[]
4,"""compostaggio: fermentazione della materia organica ottenuta con l'utilizzo di ossigeno e dei microrganismi naturalmente presenti nella stessa o dei microrganismi naturalmente presenti o selezionati durante lo svolgimento del processo stesso;""","""PDL""","""18PDL0001550_PD.xml""",[]


In [57]:
import random

from pymilvus import (
    connections,
    FieldSchema, CollectionSchema, DataType,
    Collection,
    utility
)

# This example shows how to:
#   1. connect to Milvus server
#   2. create a collection
#   3. insert entities
#   4. create index
#   5. search


_HOST = '127.0.0.1'
_PORT = '19530'

# Const names
_COLLECTION_NAME = 'demo'
_ID_FIELD_NAME = 'id_field'
_VECTOR_FIELD_NAME = 'float_vector_field'

# Vector parameters
_DIM = 128
_INDEX_FILE_SIZE = 32  # max file size of stored index

# Index parameters
_METRIC_TYPE = 'L2'
_INDEX_TYPE = 'IVF_FLAT'
_NLIST = 1024
_NPROBE = 16
_TOPK = 3


# Create a Milvus connection
def create_connection():
    print(f"\nCreate connection...")
    connections.connect(host=_HOST, port=_PORT)
    print(f"\nList connections:")
    print(connections.list_connections())


# Create a collection named 'demo'
def create_collection(name, id_field, vector_field):
    field1 = FieldSchema(name=id_field, dtype=DataType.INT64, description="int64", is_primary=True, auto_id=True)
    field2 = FieldSchema(name=vector_field, dtype=DataType.FLOAT_VECTOR, description="float vector", dim=_DIM,
                         is_primary=False)
    schema = CollectionSchema(fields=[field1, field2], description="collection description")
    collection = Collection(name=name, data=None, schema=schema)
    print("\ncollection created:", name)
    return collection


def has_collection(name):
    return utility.has_collection(name)


# Drop a collection in Milvus
def drop_collection(name):
    collection = Collection(name)
    collection.drop()
    print("\nDrop collection: {}".format(name))


# List all collections in Milvus
def list_collections():
    print("\nlist collections:")
    print(utility.list_collections())


def insert(collection, num, dim):
    data = [
        [i for i in range(num)],
        [[random.random() for _ in range(dim)] for _ in range(num)],
    ]
    collection.insert(data)
    return data[1]


def get_entity_num(collection):
    print("\nThe number of entity:")
    print(collection.num_entities)


def create_index(collection, filed_name):
    index_param = {
        "index_type": _INDEX_TYPE,
        "params": {"nlist": _NLIST},
        "metric_type": _METRIC_TYPE}
    collection.create_index(filed_name, index_param)
    print("\nCreated index:\n{}".format(collection.index().params))


def drop_index(collection):
    collection.drop_index()
    print("\nDrop index sucessfully")


def load_collection(collection):
    collection.load()


def release_collection(collection):
    collection.release()


def search(collection, vector_field, id_field, search_vectors):
    search_param = {
        "data": search_vectors,
        "anns_field": vector_field,
        "param": {"metric_type": _METRIC_TYPE, "params": {"nprobe": _NPROBE}},
        "limit": _TOPK,
        "expr": "id_field > 0"}
    results = collection.search(**search_param)
    for i, result in enumerate(results):
        print("\nSearch result for {}th vector: ".format(i))
        for j, res in enumerate(result):
            print("Top {}: {}".format(j, res))


def main():
    # create a connection
    create_connection()

    # drop collection if the collection exists
    if has_collection(_COLLECTION_NAME):
        drop_collection(_COLLECTION_NAME)

    # create collection
    collection = create_collection(_COLLECTION_NAME, _ID_FIELD_NAME, _VECTOR_FIELD_NAME)

    # show collections
    list_collections()

    # insert 10000 vectors with 128 dimension
    vectors = insert(collection, 10000, _DIM)

    # get the number of entities
    get_entity_num(collection)

    # create index
    create_index(collection, _VECTOR_FIELD_NAME)

    # load data to memory
    load_collection(collection)

    # search
    search(collection, _VECTOR_FIELD_NAME, _ID_FIELD_NAME, vectors[:3])

    # drop collection index
    drop_index(collection)

    # release memory
    release_collection(collection)

    # drop collection
    drop_collection(_COLLECTION_NAME)

In [15]:
#MILVUS_URL = "https://localhost:19530"
MILVUS_URL = "../vec_db/definitions_vectors.db"

client = MilvusClient(
    uri=MILVUS_URL
)

connections.connect(uri=MILVUS_URL)

# Define collection schema in Milvus
fields = [
    # Use auto generated id as primary key
    FieldSchema(name="id", dtype=DataType.INT64,
                is_primary=True, auto_id=True, max_length=100),
    # Store the original text to retrieve based on semantically distance
    FieldSchema(name="definition_text", dtype=DataType.VARCHAR, max_length=5000),
    FieldSchema(name="dataset", dtype=DataType.VARCHAR, max_length=10),
    FieldSchema(name="document_id", dtype=DataType.VARCHAR, max_length=40),
    FieldSchema(name="references", dtype=DataType.ARRAY, element_type=DataType.VARCHAR, max_capacity=20),
    #FieldSchema(name="sparse_vector", dtype=DataType.SPARSE_FLOAT_VECTOR),
    FieldSchema(name="dense_vector", dtype=DataType.FLOAT_VECTOR,
                dim=dense_dim),

]

schema = CollectionSchema(fields, "Definitions embeddings")

COLLECTION_NAME = "Definitions"
if utility.has_collection(COLLECTION_NAME):
    Collection(COLLECTION_NAME).drop()
collection = Collection(COLLECTION_NAME, schema, consistency_level="Strong")


#if client.has_collection(collection_name="Definitions"):
    #client.drop_collection(collection_name="Definitions")
#else:
    #client.create_collection(
        #collection_name="Definitions",
        #dimension=1024,
        #consistency_level="Strong",
        #schema=schema
    #)

In [16]:
client.get_collection_stats(COLLECTION_NAME)

{'row_count': 0}

In [17]:
# To make vector search efficient, we need to create indices for the vector fields
#sparse_index = {"index_type": "SPARSE_INVERTED_INDEX", "metric_type": "IP"}
#collection.create_index("sparse_vector", sparse_index)
dense_index = {"index_type": "AUTOINDEX", "metric_type": "IP"}
collection.create_index("dense_vector", dense_index)
collection.load()

In [18]:
# For efficiency, we insert 50 records in each small batch
for i in range(0, len(defs_list), 50):
    batched_entities = [
        #defs_list[i : i + 50],
        final_df[i: i + 50, 'definition_text'],
        final_df[i: i + 50, 'dataset'],
        final_df[i: i + 50, 'document_id'],
        final_df[i: i + 50, 'references'],
        #defs_embeddings["sparse"][i : i + 50],
        defs_embeddings["dense"][i : i + 50],
    ]
    collection.insert(batched_entities)
print("Number of entities inserted:", collection.num_entities)

Number of entities inserted: 14724


In [19]:
client.get_collection_stats(COLLECTION_NAME)

{'row_count': 14724}

In [114]:
# test query
results = collection.query(expr="", output_fields=["definition_text", 'document_id'], limit=10)

results

data: ["{'id': 454066405979222792, 'definition_text': 'georeferenziazione: tecnica di attribuzione di coordinate geografiche a un oggetto grafico, usata nelle procedure di cartografia computerizzata e nella costruzione di basi cartografiche digitali;', 'document_id': '18PDL0001470_PD.xml'}", "{'id': 454066405979222793, 'definition_text': 'sistema informativo geografico: sistema informatico, hardware e software, utilizzato per la memorizzazione e per il processamento di dati geografici nelle modalità grafica e analitica.', 'document_id': '18PDL0001470_PD.xml'}", "{'id': 454066405979222794, 'definition_text': 'gestore del sottoprodotto umido (GSU): il soggetto pubblico o privato che gestisce i sottoprodotti organici ai fini e con le modalità stabiliti dalla presente legge;', 'document_id': '18PDL0001550_PD.xml'}", "{'id': 454066405979222795, 'definition_text': 'scarti alimentari: sottoprodotti della produzione, della lavorazione e della preparazione di prodotti alimentari;', 'document_id

### Test Retrieval

In [24]:
from pymilvus import connections, Collection, MilvusClient

MILVUS_URL = "../vec_db/definitions_vectors.db"

connections.connect(
  url=MILVUS_URL
)

collection = Collection("Definitions")

from pymilvus.model.hybrid import BGEM3EmbeddingFunction

ef = BGEM3EmbeddingFunction(use_fp16=False, device="cpu")

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [25]:
connections.list_connections()

[('default', <pymilvus.client.grpc_handler.GrpcHandler at 0x7fe9ca7d03b0>),
 ('708bb6bf849b40d2a4da431c04da8b76',
  <pymilvus.client.grpc_handler.GrpcHandler at 0x7fe9d25cd160>)]

In [22]:
connections.remove_connection('default')

In [116]:
from pymilvus import (
    AnnSearchRequest,
    WeightedRanker,
)


def dense_search(col, query_dense_embedding, limit=10):
    search_params = {"metric_type": "IP", "params": {}}
    res = col.search(
        [query_dense_embedding],
        anns_field="dense_vector",
        limit=limit,
        output_fields=["definition_text"],
        param=search_params,
    )[0]
    return [hit.get("definition_text") for hit in res]


def sparse_search(col, query_sparse_embedding, limit=10):
    search_params = {
        "metric_type": "IP",
        "params": {},
    }
    res = col.search(
        [query_sparse_embedding],
        anns_field="sparse_vector",
        limit=limit,
        output_fields=["definition_text"],
        param=search_params,
    )[0]
    return [hit.get("definition_text") for hit in res]


def hybrid_search(
    col,
    query_dense_embedding,
    query_sparse_embedding,
    sparse_weight=1.0,
    dense_weight=1.0,
    limit=10,
):
    dense_search_params = {"metric_type": "IP", "params": {}}
    dense_req = AnnSearchRequest(
        [query_dense_embedding], "dense_vector", dense_search_params, limit=limit
    )
    sparse_search_params = {"metric_type": "IP", "params": {}}
    sparse_req = AnnSearchRequest(
        [query_sparse_embedding], "sparse_vector", sparse_search_params, limit=limit
    )
    rerank = WeightedRanker(sparse_weight, dense_weight)
    res = col.hybrid_search(
        [sparse_req, dense_req], rerank=rerank, limit=limit, output_fields=["definition_text"]
    )[0]
    return [hit.get("definition_text") for hit in res]


In [117]:
def doc_text_formatting(ef, query, docs):
    tokenizer = ef.model.tokenizer
    query_tokens_ids = tokenizer.encode(query, return_offsets_mapping=True)
    query_tokens = tokenizer.convert_ids_to_tokens(query_tokens_ids)
    formatted_texts = []

    for doc in docs:
        ldx = 0
        landmarks = []
        encoding = tokenizer.encode_plus(doc, return_offsets_mapping=True)
        tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"])[1:-1]
        offsets = encoding["offset_mapping"][1:-1]
        for token, (start, end) in zip(tokens, offsets):
            if token in query_tokens:
                if len(landmarks) != 0 and start == landmarks[-1]:
                    landmarks[-1] = end
                else:
                    landmarks.append(start)
                    landmarks.append(end)
        close = False
        formatted_text = ""
        for i, c in enumerate(doc):
            if ldx == len(landmarks):
                pass
            elif i == landmarks[ldx]:
                if close:
                    formatted_text += "</span>"
                else:
                    formatted_text += "<span style='color:red'>"
                close = not close
                ldx = ldx + 1
            formatted_text += c
        if close is True:
            formatted_text += "</span>"
        formatted_texts.append(formatted_text)
    return formatted_texts

In [118]:
from IPython.display import Markdown, display

query = input("Enter your search query: ")
print("Query: " + query)

query_embeddings = ef([query])

dense_results = dense_search(collection, query_embeddings["dense"][0])
#sparse_results = sparse_search(collection, query_embeddings["sparse"]._getrow(0))
#hybrid_results = hybrid_search(
    #collection,
    #query_embeddings["dense"][0],
    #query_embeddings["sparse"]._getrow(0),
    #sparse_weight=0.7,
    #dense_weight=1.0,
#)

display(Markdown("**Dense Search Results:**"))
formatted_results = doc_text_formatting(ef, query, dense_results)
for result in dense_results:
    display(Markdown(result))

#display(Markdown("\n**Sparse Search Results:**"))
#formatted_results = doc_text_formatting(ef, query, sparse_results)
#for result in formatted_results:
    #display(Markdown(result))

#display(Markdown("\n**Hybrid Search Results:**"))
#formatted_results = doc_text_formatting(ef, query, hybrid_results)
#for result in formatted_results:
    #display(Markdown(result))

Query: dog


**Dense Search Results:**

dog: means a kept animal of the Canis lupus species;

dog: means a kept animal of the Canis lupus species;

documentary check: means verification of the identification document accompanying the pet animal;

...

pet animal: means an animal of a species listed in Annex I accompanying its owner or an authorised person during non-commercial movement, and which remains for the duration of such non-commercial movement under the responsibility of the owner or the authorised person;

distress: condizione di non adattamento dell'animale a stimoli stressanti;

pet animal: means pet animal as defined in point (11) of Article 4 of Regulation (EU) 2016/429 of the European Parliament and of the Council Regulation (EU) 2016/429 of the European Parliament and of the Council of 9 March 2016 on transmissible animal diseases and amending and repealing certain acts in the area of animal health ( "Animal Health Law) ( OJ L 84, 31.3.2016, p. 1 ). ;

animale da pelliccia: le seguenti specie di animali: Cane procione (Nyctereutes procyonoides), Capra della Mongolia (Ovis Steatopyga), Castorino o Nutria (Myocastor coypus), Castoro (Castor canadensis), Cincillà (Chinchilla laniger), Coniglio o Lapin (Oryctolagus cuniculus), Coyote (Canis latrans), Donnola (Mustela nivalis), Ermellino (Mustela erminea), Foca (Phocidae), Gatto leopardo (Prionailurus bengalensis), Karakul o Astrakhan o Agnello persiano (Ovis aries platyura), Lince (Lynx), Lontra (Lutra canadensis), Marmotta (Marmota marmota), Martora (Martes martes), Moffetta o Skunk (Mephitis mephitis)), Ocelot (Felis pardalis), Ondatra o Topo muschiato (Ondatra zybethica), Opossum (Didelphis marsupialis), Procione (Procyon lotor), Puzzola (Mustela putorius), Scoiattolo (Sciurus carolinensis), Tasso (Meles meles), Visone (Mustela visori), Volpe (Vulpes vulpes), Zibellino (Martes zibellina), nonché Coccodrillo (Crocodylia), Pitone (Python), Varano (Varanus);

animals: means animals as defined in point (1) of Article 4 of Regulation (EU) 2016/429;

---